In [98]:
import pandas as pd
import numpy as np

In [99]:
df = pd.read_csv("./sales-data/Amazon Sale Report.csv")

C:\Users\peric\AppData\Local\Temp\ipykernel_10944\469468274.py:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./sales-data/Amazon Sale Report.csv")


In [100]:
df.columns

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'Unnamed: 22'],
      dtype='object')

In [101]:
df['Amount_EUR'] = df['Amount']*0.011

In [102]:

df['Date'] = pd.to_datetime(df['Date'])  # Convertir la columna de Date

# Extraer los meses disponibles
meses_disponibles = df['Date'].dt.month.unique()
meses_faltantes = [m for m in range(1, 13) if m not in meses_disponibles]

# Crear datos sintéticos para los meses faltantes
df_expansion = []
for mes in meses_faltantes:
    df_muestra = df.sample(frac=1, replace=True)  # Sampleo con reemplazo
    df_muestra = df_muestra.reset_index(drop=True)
    
    # Generar Dates en el mes correspondiente
    dias_mes = pd.date_range(start=f'2021-{mes:02d}-01', periods=30, freq='D')  # Ajusta según el año
    df_muestra['Date'] = np.random.choice(dias_mes, size=len(df_muestra), replace=True)
    
    df_expansion.append(df_muestra)

df = pd.concat([df] + df_expansion, ignore_index=True)
# Guardar el nuevo dataset con datos hasta 2020
# df_expanded.to_excel("dataset_expandido_hasta_2020.xlsx", index=False)

C:\Users\peric\AppData\Local\Temp\ipykernel_10944\3307097636.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'])  # Convertir la columna de Date


Agregar metricas que faltaban 
@NICO falta hacer que estas metricas afecten al df con los datos historicos

In [103]:
def hash_asin(asin):
    return abs(hash(asin)) % 11  # Genera un valor entre 0 y 10

# 📌 Calcular COGS Unitario (35% - 45% del precio unitario)
df["COGS Unit"] = ((df["Amount_EUR"] / df["Qty"]) * (0.35 + (df["ASIN"].apply(hash_asin) / 100))).round(2)

# 📌 Calcular COGS Total
df["COGS"] = df["COGS Unit"] * df["Qty"]

# 📌 Calcular Other Expenses (15% - 25% del Amount_EUR)
df["Other Expenses"] = (df["Amount_EUR"] * (0.15 + (df["ASIN"].apply(hash_asin) / 100))).round(2)

# 📌 Calcular Gross Profit
df["Gross Profit"] = df["Amount_EUR"] - df["COGS"]

# 📌 Calcular Net Profit
df["Net Profit"] = df["Gross Profit"] - df["Other Expenses"]

In [104]:
df.columns

Index(['index', 'Order ID', 'Date', 'Status', 'Fulfilment', 'Sales Channel ',
       'ship-service-level', 'Style', 'SKU', 'Category', 'Size', 'ASIN',
       'Courier Status', 'Qty', 'currency', 'Amount', 'ship-city',
       'ship-state', 'ship-postal-code', 'ship-country', 'promotion-ids',
       'B2B', 'fulfilled-by', 'Unnamed: 22', 'Amount_EUR', 'COGS Unit', 'COGS',
       'Other Expenses', 'Gross Profit', 'Net Profit'],
      dtype='object')

In [105]:
df.describe()

,index,Date,Qty,Amount,ship-postal-code,Amount_EUR,COGS Unit,COGS,Other Expenses,Gross Profit,Net Profit
count,1.160775e+06,1160775,1.160775e+06,1.090416e+06,1.160463e+06,1.090416e+06,1090416.00,1.044500e+06,1.090416e+06,1.044500e+06,1.044500e+06
mean,6.447466e+04,2021-09-01 09:18:13.076953856,9.046353e-01,6.485350e+02,4.640678e+05,7.133885e+00,inf,2.858907e+00,1.426766e+00,4.288021e+00,2.858510e+00
min,0.000000e+00,2021-01-01 00:00:00,0.000000e+00,0.000000e+00,1.100010e+05,0.000000e+00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.224800e+04,2021-07-08 00:00:00,1.000000e+00,4.490000e+02,3.824210e+05,4.939000e+00,1.98,1.960000e+00,9.600000e-01,2.947000e+00,1.920000e+00
50%,6.444200e+04,2021-09-15 00:00:00,1.000000e+00,6.050000e+02,5.000330e+05,6.655000e+00,2.72,2.650000e+00,1.290000e+00,4.019000e+00,2.629000e+00
75%,9.671900e+04,2021-11-23 00:00:00,1.000000e+00,7.880000e+02,6.000240e+05,8.668000e+00,3.64,3.510000e+00,1.770000e+00,5.259000e+00,3.601000e+00
max,1.289740e+05,2022-06-29 00:00:00,1.500000e+01,5.584000e+03,9.898980e+05,6.142400e+01,inf,2.295000e+01,1.088000e+01,3.870400e+01,2.826400e+01
std,3.722917e+04,NaN,3.119641e-01,2.810760e+02,1.914617e+05,3.091836e+00,NaN,1.277435e+00,6.760733e-01,1.869412e+00,1.317848e+00


In [106]:
df.drop(columns=["index", "Unnamed: 22"], inplace=True)

In [107]:
df['Amount_EUR'].max()

61.424

### Agregador de Data Historica

In [108]:
min_date = df['Date'].min()
max_date = df['Date'].max()
print(f"Rango de fechas: {min_date} - {max_date}")

Rango de fechas: 2021-01-01 00:00:00 - 2022-06-29 00:00:00


### Descargar Archivo

In [109]:
df.to_csv("./sales-data/amazon-sale-report-clean.csv", index=False)

In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1160775 entries, 0 to 1160774
Data columns (total 28 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   Order ID            1160775 non-null  object        
 1   Date                1160775 non-null  datetime64[ns]
 2   Status              1160775 non-null  object        
 3   Fulfilment          1160775 non-null  object        
 4   Sales Channel       1160775 non-null  object        
 5   ship-service-level  1160775 non-null  object        
 6   Style               1160775 non-null  object        
 7   SKU                 1160775 non-null  object        
 8   Category            1160775 non-null  object        
 9   Size                1160775 non-null  object        
 10  ASIN                1160775 non-null  object        
 11  Courier Status      1099163 non-null  object        
 12  Qty                 1160775 non-null  int64         
 13  currency    

In [111]:
df['Date']

0         2022-04-30
1         2022-04-30
2         2022-04-30
3         2022-04-30
4         2022-04-30
             ...    
1160770   2021-12-04
1160771   2021-12-05
1160772   2021-12-11
1160773   2021-12-20
1160774   2021-12-14
Name: Date, Length: 1160775, dtype: datetime64[ns]